It needs to be re-written before SSA as the entire notebook is completed through copilot and ChatGpt o4-mini.

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [ ]:
# a mapping object
# returns a dict with user's environmental vars as key and values

os.environ["openblas_num"] = "1" # stop basic lin algebra subprograms from over-parallelising
os.environ["omp_num_threads"] = "1" # again single-threaded open multi-processing

base_dir = Path().resolve().parent
dat_dir = base_dir / "data"
burg_dir = dat_dir / "repo-dat"
pp_dir = dat_dir / "lr-pp" # https://www.gov.uk/government/collections/price-paid-data

def read_csv_chunkwise(files, **kwargs):
    dfs = []
    for f in files:
        dfs.append(pd.read_csv(f, **kwargs))
    return pd.concat(dfs, ignore_index=True)

In [15]:
street_dat = list(burg_dir.glob("*-metropolitan-street.csv"))
outcome_dat = list(burg_dir.glob("*-metropolitan-outcomes.csv"))
search_dat = list(burg_dir.glob("*-metropolitan-stop-and-search.csv"))

print("Street files:", street_dat)
print("Outcome files:", outcome_dat)
print("Search files:", search_dat)

Street files: [PosixPath('/Users/youjungum/dc2/data/repo-dat/2021-06-metropolitan-street.csv'), PosixPath('/Users/youjungum/dc2/data/repo-dat/2016-04-metropolitan-street.csv'), PosixPath('/Users/youjungum/dc2/data/repo-dat/2023-04-metropolitan-street.csv'), PosixPath('/Users/youjungum/dc2/data/repo-dat/2018-06-metropolitan-street.csv'), PosixPath('/Users/youjungum/dc2/data/repo-dat/2014-06-metropolitan-street.csv'), PosixPath('/Users/youjungum/dc2/data/repo-dat/2012-01-metropolitan-street.csv'), PosixPath('/Users/youjungum/dc2/data/repo-dat/2014-09-metropolitan-street.csv'), PosixPath('/Users/youjungum/dc2/data/repo-dat/2018-09-metropolitan-street.csv'), PosixPath('/Users/youjungum/dc2/data/repo-dat/2013-11-metropolitan-street.csv'), PosixPath('/Users/youjungum/dc2/data/repo-dat/2021-09-metropolitan-street.csv'), PosixPath('/Users/youjungum/dc2/data/repo-dat/2013-02-metropolitan-street.csv'), PosixPath('/Users/youjungum/dc2/data/repo-dat/2015-05-metropolitan-street.csv'), PosixPath('/U

In [16]:
burg_street = pd.concat(
    (pd.read_csv(f, parse_dates=["Month"], dtype={"LSOA code": str})
     for f in street_dat),
    ignore_index=True
) if street_dat else pd.DataFrame()

In [17]:
burg_outcome = pd.concat(
    (pd.read_csv(f, parse_dates=["Month"], dtype={"LSOA code": str})
     for f in outcome_dat),
    ignore_index=True
) if outcome_dat else pd.DataFrame()

In [19]:
burg_search = pd.concat(
    (pd.read_csv(f, parse_dates=["Date"], dtype={"Latitude": float})
     for f in search_dat),
    ignore_index=True
) if search_dat else pd.DataFrame()

In [21]:
df_burg = (
    burg_street[burg_street["Crime type"] == "Burglary"]
    .assign(Year=lambda d: d["Month"].dt.year)
    .groupby(["LSOA code", "Year"])
    .size()
    .reset_index(name="BurglaryCount")
    .rename(columns={"LSOA code": "LSOA_code"})
)

In [24]:
pp_dir = dat_dir / "lr-pp"
pp_dat = list(pp_dir.glob("*.csv"))

pp_cols = [
    "TransactionID", "Price Paid", "Date of Transfer", "Postcode",
    "Property Type", "Old/New", "Duration",
    "PAON", "SAON", "Street", "Locality", "Town/City",
    "District", "County", "PPD Category Type", "Record Status"
]

frames = []

for f in pp_dat:
    tmp_pp = pd.read_csv(
        f, 
        header=None, 
        names=pp_cols, 
        parse_dates=[2], 
        dayfirst=False,
        dtype={"Postcode": str}, 
        na_values = ["", " "]
    ) 
    frames.append(tmp_pp)

df_pp_raw = pd.concat(frames, ignore_index=True)

In [26]:
df_pp = (
    df_pp_raw[df_pp_raw["County"].str.contains("LONDON", na=False)]
    .assign(Year=lambda d: d["Date of Transfer"].dt.year) 
    .groupby(["Postcode", "Year"]) 
    .agg(
        MedianPrice = ("Price Paid", "median"), 
        NTransactions = ("Price Paid", "count"),
    )
    .reset_index()
)

print("df_pp:", df_pp.shape)

df_pp: (412664, 4)


In [27]:
burg_borough = (
    burg_street[burg_street["Crime type"]=="Burglary"]
    .assign(Year=lambda d: d["Month"].dt.year)
    .groupby(["Falls within","Year"])
    .size()
    .reset_index(name="BurglaryCount")
    .rename(columns={"Falls within":"Borough"})
)

In [ ]:
# ONS Postcode Directory (UK Government)
# https://open-geography-portalx-ons.hub.arcgis.com/datasets/ons::ons-postcode-directory-august-2024-for-the-uk/about
onspd_dir = base_dir / "data" / "onspd-aug24" / "ONSPD_AUG_2024_UK.csv"
print(onspd_dir.exists())

True


In [42]:
if onspd_dir.exists():
    onspd = pd.read_csv(
        onspd_dir, 
        usecols=["lsoa11", "pcds", "oslaua", "osward"], 
        dtype=str
    ).rename(columns={
        "lsoa11": "LSOACode", 
        "pcds": "PostcodeCleaned", 
        "oslaua": "BoroughCode", 
        "osward": "ONSWardCode"
    })
    print("ONSPD: ", len(onspd))
else: 
    raise FileNotFoundError(f"ONSPD csv file could not be found at {onspd_dir}")

ONSPD:  2706710


In [43]:
df_burg_raw = burg_street.rename(columns={"LSOA code": "LSOACode"})

df_burg_raw = df_burg_raw.merge(
    onspd, 
    on="LSOACode", 
    how="left"
)

print(df_burg_raw[["LSOACode", "BoroughCode"]].drop_duplicates().head())

: 

In [ ]:
df_burg_borough = (
    df_burg_raw[df_burg_raw["Crime type"]=="Burglary"]
    .assign(Year = lambda d: d["Month"].dt.year)
    .groupby(["BoroughCode","Year"])
    .size()
    .reset_index(name="BurglaryCount")
)

df_burg_borough.head()

In [ ]:
df_pp["Postcode_clean"] = (
    df_pp["Postcode"]
    .str.replace(r"\s+", "", regex=True)
    .str.upper()
)
df_pp = df_pp.merge(onspd, on="Postcode_clean", how="left")
df_pp_borough = (
    df_pp
    .groupby(["Borough","Year"])
    .agg(
        MedianPrice   = ("MedianPrice","median"),
        NTransactions = ("NTransactions","sum")
    )
    .reset_index()
)
